In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Import modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import cv2

#keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
import sklearn.metrics as metrics

In [3]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

In [4]:
mnist = pd.read_csv("../input/mnist-in-csv/mnist_train.csv")
atoz = pd.read_csv("../input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv")
print("mnist: %s, atoz: %s, " %(mnist.shape, atoz.shape))

In [5]:
mnist.head()

In [6]:
mnist.label.unique()

In [7]:
atoz['0'].unique()

In [8]:
atoz['0']+=10

In [9]:
mnist = mnist.sample(frac = 1)

In [10]:
atoz.head()

In [11]:
atoz = atoz.sample(frac = 1)

In [12]:
atoz.head()

In [13]:
train = pd.DataFrame( np.concatenate( (mnist.values, atoz.values), axis=0 ) )
del mnist
del atoz

In [14]:
train.head()

In [15]:
train.head(10)

In [16]:
train.shape

In [17]:
print(train[0].unique())

In [18]:
# Constants
HEIGHT = 28
WIDTH = 28

In [19]:
# Split x and y
train_x = train.iloc[:,1:]
train_y = train.iloc[:,0]
del train


In [20]:
print(train_x.shape,train_y.shape)

In [21]:
def reshape(image):
    image = image.reshape([HEIGHT, WIDTH])
    # image = np.fliplr(image)
    # image = np.rot90(image)
    return image

In [22]:
# Flip and rotate image
train_x = np.asarray(train_x)
train_x = train_x.reshape(-1,28,28,1)
print ("train_x:",train_x.shape)

In [23]:
mapp={0:'0',1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',10:'A',11:'B',12:'C',13:'D',14:'E',15:'F',16:'G',17:'H',18:'I',19:'J',20:'K',21:'L',22:'M',23:'N',24:'O',25:'P',26:'Q',27:'R',28:'S',29:'T',30:'U',31:'V',32:'W',33:'X',34:'Y',35:'Z'}

In [24]:
# plot image
plt.imshow(train_x[0], cmap=plt.get_cmap('gray'))
plt.title(mapp[train_y[0]])

In [25]:
train_y[0]

In [26]:
# plot image
for i in range(100, 109):
    plt.subplot(330 + (i+1))
    plt.imshow(train_x[i], cmap=plt.get_cmap('gray'))
    plt.title(mapp[train_y[i]])

In [27]:
# Normalise
train_x = train_x.astype('float32')
train_x /= 255

In [28]:
# number of classes
num_classes = train_y.nunique()

In [29]:
print(num_classes)

In [30]:
# One hot encoding
train_y = np_utils.to_categorical(train_y, num_classes)
print("train_y: ", train_y.shape)

In [31]:
print("train: ",train_x.shape )

In [32]:
# partition to train and val
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size= 0.20, random_state=7)

In [33]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import numpy as np

In [34]:
#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=34, input_shape=(28,28,1), kernel_size=(11,11), strides=(2,2), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=150, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=200, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=300, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=200, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(28,28,1,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.3))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.3))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.3))

#Output Layer
AlexNet.add(Dense(num_classes))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()


# # Building model
# # ((Si - Fi + 2P)/S) + 1
# model = Sequential()

# model.add(Conv2D(filters=128, kernel_size=(5,5), padding = 'same', activation='relu',\
#                  input_shape=(HEIGHT, WIDTH,1)))
# model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# model.add(Conv2D(filters=64, kernel_size=(3,3) , padding = 'same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Flatten())
# model.add(Dense(units=128, activation='relu'))
# model.add(Dropout(.5))
# model.add(Dense(units=num_classes, activation='softmax'))

# model.summary()

In [35]:
AlexNet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
MCP = ModelCheckpoint('Best_points.h5',verbose=1,save_best_only=True,monitor='val_accuracy',mode='max')
ES = EarlyStopping(monitor='val_accuracy',min_delta=0,verbose=0,patience=3,mode='max')
RLP = ReduceLROnPlateau(monitor='val_loss',patience=3,factor=0.2,min_lr=0.0001)

In [37]:
history = AlexNet.fit(train_x, train_y, epochs=50, batch_size=512, verbose=1, \
                    validation_data=(val_x, val_y),callbacks=[MCP,ES,RLP])

In [38]:
# plot accuracy and loss
def plotgraph(epochs, acc, val_acc,title):
    # Plot training & validation accuracy values
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Model accuracy')
    plt.ylabel(title)
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [39]:
#%%
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)

In [40]:
# Accuracy curve
plotgraph(epochs, acc, val_acc,"accuracy")

In [41]:
# loss curve
plotgraph(epochs, loss, val_loss,"loss")